連立一次方程式をPythonの数理最適化モデルで解く

最大化したい問題ではないので、「目的関数がゼロ」の最適化問題

In [ ]:
import pulp
# 数理モデルの定義（最大化問題になっているが、
problem = pulp.LpProblem("SLE", pulp.LpMaximize)
problem

SLE:
MAXIMIZE
None
VARIABLES

In [ ]:
# 変数の定義
x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

In [ ]:
# 制約条件の定義：==なので等式制約（目的関数ではない）
problem += 120*x + 150*y == 1440
problem += x + y == 10
problem

SLE:
MAXIMIZE
None
SUBJECT TO
_C1: 120 x + 150 y = 1440

_C2: x + y = 10

VARIABLES
x free Continuous
y free Continuous

In [ ]:
# 問題を解く
status = problem.solve()     # 解が見つかったかどうかをsttatusに入れる
print("Status : ", pulp.LpStatus[status])

Status :  Optimal


In [6]:
# 最適化結果の表示
print(f"x : {x.value()}, y : {y.value()}")

x : 2.0, y : 8.0


$製品 m を x\:kg、製品 n を y\:kg作る問題 \\
【制約条件】 \\
x + 3y \leq 30 （原料の制約）\\
2x + y \leq 40 （原料の制約）\\
x \geq 0 , y \geq 0 \\
x + 2y （利益を最大化する）
$

In [8]:
import pulp

# 問題定義
problem = pulp.LpProblem("Profit_Maximize", pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable("x", lowBound=0, cat="Continuous")
y = pulp.LpVariable("y", lowBound=0, cat="Continuous")

# 目的関数の設定
problem += x + 2*y, "Total_Profit"

# 制約条件の追加
problem += x + 3*y <= 30
problem += 2*x + y <= 40
problem += x >= 0
problem += y >= 0

# 問題を解く
status = problem.solve()

# 結果の表示
print(f"Status : {pulp.LpStatus[status]}")
print("---Optimal Solution---")
print(f"製品 x : {x.value()} kg")
print(f"製品 y : {y.value()} kg")
print(f"合計利益：{pulp.value(problem.objective)}")

Status : Optimal
---Optimal Solution---
製品 x : 18.0 kg
製品 y : 4.0 kg
合計利益：26.0


さらに規模の大きな数理最適化問題を解く

In [9]:
import pandas as pd
import pulp

# データの取得
stock_df = pd.read_csv("stocks.csv")
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [10]:
# データの取得
requires_df = pd.read_csv("requires.csv")
requires_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [11]:
# データの取得
gains_df = pd.read_csv("gains.csv")
gains_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [15]:
# 製品リストを作成
P = gains_df["p"].tolist()
P

['p1', 'p2', 'p3', 'p4']

In [16]:
# 原料リストを作成
M = stock_df["m"].tolist()
M

['m1', 'm2', 'm3']

In [ ]:
# 定数の定義
# 各原料の在庫量辞書：stock（stock_dfから取り出して、辞書形式にする）
stock = {row.m:row.stock for row in stock_df.itertuples()}
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [ ]:
# 各製品の利益辞書：gain（gain_dfから取り出して、辞書形式にする）
gain = {row.p:row.gain for row in gains_df.itertuples()}
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

In [19]:
# 各製品を作るのに必要な原料辞書：reqire（requires_dfから取り出して、辞書形式にする）
require = {(row.p, row.m):row.require for row in requires_df.itertuples()}
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [20]:
# 数理最適化モデルの定義
problem = pulp.LpProblem("LP2", pulp.LpMaximize)
problem

LP2:
MAXIMIZE
None
VARIABLES

In [ ]:
# 変数の定義
x = pulp.LpVariable.dicts("x", P, cat="Continuous")     # 各製品ごとの生産量x[p]を定義。
x

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}

In [33]:
# 制約条件
# 生産量は０以上
for p in P:
    problem += x[p] >= 0

# 生産量は在庫の範囲内（必要な原料×生産量<=在庫）
for m in M: # 各原料ごとに、全製品が使う総使用量が在庫量を超えないように制限。
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

In [34]:
# 目的関数の定義
problem += pulp.lpSum(gain[p] * x[p] for p in P) # 製品ごとの、利益×生産量 を合計して最大化する。
problem

c:\Users\kondoy\AppData\Local\miniconda3\envs\python312_Math_Opti_TextBook\Lib\site-packages\pulp\pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


LP2:
MAXIMIZE
3*x_p1 + 4*x_p2 + 4*x_p3 + 5*x_p4 + 0.0
SUBJECT TO
_C1: x_p1 >= 0

_C2: x_p2 >= 0

_C3: x_p3 >= 0

_C4: x_p4 >= 0

_C5: 2 x_p1 + 3 x_p2 + 2 x_p4 <= 35

_C6: 2 x_p2 + 2 x_p3 + 2 x_p4 <= 22

_C7: x_p1 + 2 x_p3 + 2 x_p4 <= 27

_C8: x_p1 >= 0

_C9: x_p2 >= 0

_C10: x_p3 >= 0

_C11: x_p4 >= 0

_C12: 2 x_p1 + 3 x_p2 + 2 x_p4 <= 35

_C13: 2 x_p2 + 2 x_p3 + 2 x_p4 <= 22

_C14: x_p1 + 2 x_p3 + 2 x_p4 <= 27

VARIABLES
x_p1 free Continuous
x_p2 free Continuous
x_p3 free Continuous
x_p4 free Continuous

In [35]:
# 問題を解く
status = problem.solve()
print(f"Status : {pulp.LpStatus[status]}")

Status : Optimal


In [36]:
# 計算結果の可視化
for p in P:
    print(f"{p}の生産量：{x[p].value()}")

print(f"最大利益：{problem.objective.value()}")

p1の生産量：12.142857
p2の生産量：3.5714286
p3の生産量：7.4285714
p4の生産量：0.0
最大利益：80.42857099999999


In [ ]:
import pandas as pd     # CSVからデータをデータフレームで読み込むために必要
import pulp

# データの取得
stock_df = pd.read_csv("stocks.csv")
# データの取得
requires_df = pd.read_csv("requires.csv")
# データの取得
gains_df = pd.read_csv("gains.csv")

# 製品リストを作成
P = gains_df["p"].tolist()  # gains_dfのp列をリストに変換してPに入れる
# 原料リストを作成
M = stock_df["m"].tolist()

# 定数の定義（データフレームの内容を辞書にまとめる）
# 各原料の在庫量辞書：stock（stock_dfから（原料名, 在庫量）のタプルで1行ずつ取り出して、辞書形式にまとめる）
stock = {row.m:row.stock for row in stock_df.itertuples()}

# 各製品の利益辞書：gain（gain_dfから取り出して、辞書形式にする）
gain = {row.p:row.gain for row in gains_df.itertuples()}

# 各製品を作るのに必要な原料辞書：reqire（requires_dfから取り出して、辞書形式にする）
require = {(row.p, row.m):row.require for row in requires_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem("LP2", pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts("x", P, cat="Continuous")

# 制約条件
# 生産量は０以上
for p in P:
    problem += x[p] >= 0

# 生産量は在庫の範囲内（必要な原料×生産量<=在庫）
for m in M:
    problem += pulp.lpSum(require[p,m] * x[p] for p in P) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum(gain[p] * x[p] for p in P)
problem

# 問題を解く
status = problem.solve()
print(f"Status : {pulp.LpStatus[status]}")

# 計算結果の可視化
for p in P:
    print(f"{p}の生産量：{x[p].value()}")

print(f"最大利益：{problem.objective.value()}")

Status : Optimal
p1の生産量：12.142857
p2の生産量：3.5714286
p3の生産量：7.4285714
p4の生産量：0.0
最大利益：80.42857099999999


### 整数計画問題に変更する
生産量が少数（実数）になっている。

生産量が【個数】だった場合、ただ少数を丸めた結果（76万円）が最適解とは限らない。

$\underline{変更箇所}$
-   変数の定義でContinuous---→Integer に変更する。
-   モデル名をInteger Programmingの略、IPに変更する。

整数計画法に変更することで、線形計画法の結果をただ丸めるよりも、利益が大きくなる（79万円）。

In [37]:
import pandas as pd     # CSVからデータをデータフレームで読み込むために必要
import pulp

# データの取得
stock_df = pd.read_csv("stocks.csv")
requires_df = pd.read_csv("requires.csv")
gains_df = pd.read_csv("gains.csv")

# 製品リストを作成
P = gains_df["p"].tolist()  # gains_dfのp列をリストに変換してPに入れる
M = stock_df["m"].tolist()  # 原料リストを作成

# 定数の定義（データフレームの内容を辞書にまとめる）
stock = {row.m:row.stock for row in stock_df.itertuples()}                 # 各原料の在庫量辞書：stock（stock_dfから（原料名, 在庫量）のタプルで1行ずつ取り出して、辞書形式にまとめる）
gain = {row.p:row.gain for row in gains_df.itertuples()}                   # 各製品の利益辞書：gain（gain_dfから取り出して、辞書形式にする）
require = {(row.p, row.m):row.require for row in requires_df.itertuples()} # 各製品を作るのに必要な原料辞書：reqire（requires_dfから取り出して、辞書形式にする）

# 数理最適化モデルの定義
problem = pulp.LpProblem("IP", pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts("x", P, cat="Integer")

# 制約条件
# 生産量は０以上
for p in P:
    problem += x[p] >= 0

# 生産量は在庫の範囲内（必要な原料×生産量<=在庫）
for m in M:
    problem += pulp.lpSum(require[p,m] * x[p] for p in P) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum(gain[p] * x[p] for p in P)
problem

# 問題を解く
status = problem.solve()
print(f"Status : {pulp.LpStatus[status]}")

# 計算結果の可視化
for p in P:
    print(f"{p}の生産量：{x[p].value()}")

print(f"最大利益：{problem.objective.value()}")

Status : Optimal
p1の生産量：13.0
p2の生産量：3.0
p3の生産量：7.0
p4の生産量：-0.0
最大利益：79.0
